In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_core.documents import Document
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate, LLMChain

In [2]:
loader= PyPDFDirectoryLoader("pdfs")

In [3]:
 
data=loader.load()

In [4]:
data

[Document(metadata={'source': 'pdfs\\TheHistoryofApple1.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/364326882\nThe History of Apple\nArticle · October 2022\nCITATIONS\n0\nREADS\n29,507\n2 authors, including:\nOdina Kamolova\nWestminster International University in Tashkent\n4 PUBLICATIONS\xa0\xa0\xa01 CITATION\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Odina Kamolova on 14 October 2022.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'pdfs\\TheHistoryofApple1.pdf', 'page': 1}, page_content='IJESC, September 2022                                                                        29800                              http:// ijesc.org/ \n \nISSN 2321 3361 © 2022 IJESC  \n  \n \n \nThe History of Apple \nMuxtorova Nasiba1, Kamolova Odina2 \nUniveristy of Westminster, \nTASHKENT Uzbekistan. \n \nIntroduction:    \nApp

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=15)
text_chunk = text_splitter.split_documents(data)

In [ ]:
emdedding= OllamaEmbeddings(model="llama3.2:1b" )

In [ ]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=emdedding,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
uuids = [str(uuid4()) for _ in range(len(text_chunk))]
 
vector_store.add_documents(documents=text_chunk, ids=uuids)

In [ ]:
results = vector_store.similarity_search(
    "WHO IS STEVE JOBS",
    k=2,
)

In [ ]:
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
from langchain_ollama import OllamaLLM

In [ ]:
llm=OllamaLLM(model="llama3.2:1b")

In [ ]:
template = """
You are an expert assistant specializing in Apple's history. Your job is to answer questions accurately using the provided context. 
If the context doesn't contain enough information, ask the user to provide additional details. Never make up answers.
 
### Context:
{context}
 
### User Query:
{query}
 
### Your Response:
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template,
)

In [ ]:
apple_history_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
query = "When was Apple founded, and who were the founders?"

In [ ]:
response = apple_history_chain.run(context=results, query=query)

In [ ]:
print(response)